## 1.0 Mathematical Addition using RNN

## 2.0 Generate Dataset

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [2]:
DIGITS = 3
TRAINING_SIZE = 50000
MAXLEN = 2*DIGITS + 1

In [3]:
questions = []
answers = []

In [6]:
seen = set()
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))for i in range(np.random.randint(1, DIGITS +1))))
    
    a , b= f(), f()
    
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
 
    seen.add(key)
    
    q = '{}+{}'.format(a,b)
    question = q + ' ' * (MAXLEN - len(q))
    
    a = str(a + b)
    answer = a + ' ' * (DIGITS +1 - len(a))
    
    questions.append(question)
    answers.append(answer)
    